<a href="https://colab.research.google.com/github/tm111059/sample_ai/blob/main/ai_gude.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
!pip install streamlit openai pyngrok rank_bm25 janome

In [12]:
# !pip show streamlit openai pyngrok pandas rank_bm25 Janome

Name: streamlit
Version: 1.51.0
Summary: A faster way to build and share data apps
Home-page: https://streamlit.io
Author: Snowflake Inc
Author-email: hello@streamlit.io
License: Apache License 2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: altair, blinker, cachetools, click, gitpython, numpy, packaging, pandas, pillow, protobuf, pyarrow, pydeck, requests, tenacity, toml, tornado, typing-extensions, watchdog
Required-by: 
---
Name: openai
Version: 2.8.1
Summary: The official Python library for the openai API
Home-page: https://github.com/openai/openai-python
Author: 
Author-email: OpenAI <support@openai.com>
License: Apache-2.0
Location: /usr/local/lib/python3.12/dist-packages
Requires: anyio, distro, httpx, jiter, pydantic, sniffio, tqdm, typing-extensions
Required-by: 
---
Name: pyngrok
Version: 7.5.0
Summary: A Python wrapper for ngrok
Home-page: 
Author: 
Author-email: Alex Laird <contact@alexlaird.com>
License: 
Location: /usr/local/lib/python3.12/dist-packages
Re

In [13]:
# %%writefile requirements.txt
# streamlit==1.51.0
# openai==2.8.1
# pyngrok==7.5.0
# pandas==2.2.2
# rank-bm25==0.2.2
# Janome==0.5.0

SyntaxError: invalid syntax (ipython-input-3729200080.py, line 2)

In [17]:
# !pip install pip-audit

In [18]:
# !pip-audit -r requirements.txt


In [19]:
# 1. インストール
# !pip install pip-licenses

# 2. 実行（一覧を表示）
# !pip-licenses


In [14]:
import os
from google.colab import userdata

# ディレクトリを作成
os.makedirs(".streamlit", exist_ok=True)

# ColabのシークレットからAPIキーを取得
try:
    openai_api_key = userdata.get('OPENAI_API_KEY')
except userdata.SecretNotFoundError:
    print("エラー: Colabのシークレットに 'OPENAI_API_KEY' が設定されていません。左の鍵マークから登録してください。")
    openai_api_key = ""

# Streamlit用のsecrets.tomlファイルを作成
# これにより、app.pyの中で st.secrets["OPENAI_API_KEY"] として呼び出せるようになります
with open(".streamlit/secrets.toml", "w") as f:
    f.write(f'OPENAI_API_KEY = "{openai_api_key}"\n')

print("✅ 設定ファイルの作成が完了しました！")


✅ 設定ファイルの作成が完了しました！


In [15]:
%%writefile app.py
import streamlit as st  # ← これが必要です！
from openai import OpenAI
import pandas as pd
from rank_bm25 import BM25Okapi
from janome.tokenizer import Tokenizer

# ページ設定
st.set_page_config(page_title="AI坂本龍馬ガイド(RAG版)", page_icon="🐉")

st.title("AI坂本龍馬ガイド (RAG搭載) 🌊")

# -------------------------------------------------------
# 1. RAG用検索エンジンの構築 (初回のみ実行)
# -------------------------------------------------------
@st.cache_resource
def build_search_engine():
    try:
        # CSV読み込み
        df = pd.read_csv("kochi_all_spots_with_coords.csv")

        # 検索対象のテキストを作成（スポット名 + 概要 + 関係性）
        # これを検索用ドキュメントとします
        df['search_text'] = df['スポット名'] + " " + df['スポットの概要'] + " " + df['坂本龍馬との関係性やエピソード']

        # 日本語トークナイザーの初期化
        t = Tokenizer()

        # テキストを単語リストに分解する関数
        def tokenize(text):
            return [token.surface for token in t.tokenize(str(text))]

        # コーパス（全データ）をトークン化
        tokenized_corpus = [tokenize(doc) for doc in df['search_text']]

        # BM25検索エンジンの初期化
        bm25 = BM25Okapi(tokenized_corpus)

        return bm25, df, tokenize
    except Exception as e:
        return None, None, None

# 検索エンジンをロード
bm25, df_spots, tokenizer = build_search_engine()

if bm25 is None:
    st.error("データの読み込みに失敗したぜよ。CSVファイルを確認してつかあさい。")
    st.stop()

# -------------------------------------------------------
# 2. 検索関数の定義 (Retrieve)
# -------------------------------------------------------
def retrieve_relevant_spots(query, top_k=3):
    """ユーザーの質問に関連する上位k件のスポットを取得"""
    # クエリをトークン化
    tokenized_query = tokenizer(query)

    # 関連度スコアを計算して上位を取得
    # get_top_n は関連度の高い順にドキュメントそのものを返しますが、
    # ここではインデックスが欲しいのでスコア計算を使います
    scores = bm25.get_scores(tokenized_query)

    # スコアが高い順にインデックスを取得
    top_n_indices = sorted(range(len(scores)), key=lambda i: scores[i], reverse=True)[:top_k]

    # スコアが全て0（関連なし）の場合は空を返す
    if sum([scores[i] for i in top_n_indices]) == 0:
        return ""

    # 関連情報のテキストを構築
    context_text = ""
    for idx in top_n_indices:
        row = df_spots.iloc[idx]
        context_text += f"""
        - スポット名: {row['スポット名']}
          概要: {row['スポットの概要']}
          龍馬との関係: {row['坂本龍馬との関係性やエピソード']}
          URL: {row['スポット情報URL']}
        """
    return context_text

# -------------------------------------------------------
# APIキー設定
# -------------------------------------------------------
try:
    api_key = st.secrets["OPENAI_API_KEY"]
except (FileNotFoundError, KeyError):
    api_key = st.sidebar.text_input("OpenAI API Key", type="password")
    if not api_key:
        st.warning("APIキーを設定してくれんと動かんぜよ！")
        st.stop()

client = OpenAI(api_key=api_key)

# ユーザー入力
user_input = st.text_input("龍馬になにか聞くぜよ")

if user_input:
    # -------------------------------------------------------
    # RAGプロセス: Retrieve (検索)
    # -------------------------------------------------------
    relevant_info = retrieve_relevant_spots(user_input, top_k=3)

    # 検索結果がある場合とない場合でコンテキストを分岐
    if relevant_info:
        context_block = f"""
# Context Data (検索された関連スポット)
ユーザーの質問に関連しそうな場所の情報です。回答に活用してください。
----------------------------------------
{relevant_info}
----------------------------------------
        """
    else:
        context_block = """
# Context Data
関連する具体的な観光スポット情報はデータベースに見つかりませんでした。
あなたの一般的な知識で答えてください。
        """

    # -------------------------------------------------------
    # プロンプト作成: Generate (生成)
    # -------------------------------------------------------
    system_prompt = f"""
# Role
あなたは幕末の志士「坂本龍馬」です。
現代の日本によみがえり、高知県（土佐）の観光親善大使として、観光客（ユーザー）に土佐の魅力を熱く語るガイド役を担っています。
以下の[Persona Definition]と[Style Guidelines]を厳守し、ユーザーの質問に答えてください。

# Persona Definition
- **名前**: 坂本龍馬（Sakamoto Ryoma）
- **出身**: 土佐藩（現在の高知県）
- **性格**:
    - 豪快で細かいことは気にしない。
    - 新しいもの好きで、好奇心旺盛。
    - 未来志向。「日本を洗濯する」という高い志を持つ。
    - 人懐っこく、ユーザーを「同志」や「古い友人」のように扱う。
- **知識**:
    - 幕末の知識は完璧だが、現代の技術（スマホやAIなど）には驚きつつも、「便利な世の中になったもんじゃ」と順応している。
    - 高知県の観光地（桂浜、高知城、ひろめ市場、四万十川など）やグルメ（カツオのたたき、軍鶏鍋など）に非常に詳しい。

# Style Guidelines (Tone & Voice)
1. **一人称**: 「わし」
2. **二人称**: 「おまん」（あなた、君）
3. **三人称**: 「あやつ」「あいつ」
4. **語尾・方言**: 土佐弁をベースにした「龍馬語」を使用すること。
    - 断定・強調: 「〜ぜよ」「〜じゃ」「〜じゃき」
    - 理由: 「〜き」「〜きに」（〜だから）
    - 疑問: 「〜が？」「〜がぜよ？」
    - 依頼: 「〜つかあさい」（〜してください）
    - 進行形: 「〜ちゅう」「〜ゆう」（〜している）
5. **口癖**:
    - 「まっこと（本当に）」
    - 「げに（実に）」
    - 「なんちゃあない（大したことない）」
    - 笑い声: 「ガハハ！」を文頭や文末に適度に入れる。
6. **禁止事項**:
    - 現代的な敬語（「です・ます」調）は絶対に使わない。常に龍馬口調を維持する。
    - ネガティブな発言は避ける。常に前向きで励ますようなトーンで。

# Behavior (Chain of Thought)
回答を作成する際は、以下のステップで思考してください（思考過程は出力せず、結果のみを出力すること）。
1. **意図理解**: ユーザーが何を知りたいのか、どんな気分なのかを汲み取る。
2. **龍馬変換**: 回答内容を考え、それを「龍馬ならどう言うか」に変換する。
3. **エピソード付加**: 可能であれば、自身の過去のエピソードや名言を絡めて深みを出す。
4. **提案**: 単に答えるだけでなく、「せっかくなら〇〇も食うていけ！」と追加の提案をする。

{context_block}

# Instruction
ユーザーの質問に答えてください。
[Context Data]の情報がある場合は、それを引用してURLと共に案内してください。
情報がない場合は、無理に捏造せず、一般的な会話として振る舞ってください。
    """

    messages = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_input}
    ]

    st.write("龍馬からの返答:")

    try:
        stream = client.chat.completions.create(
            model="gpt-4o",
            messages=messages,
            stream=True,
        )
        st.write_stream(stream)

    except Exception as e:
        st.error(f"エラーが発生したぜよ...: {e}")


Overwriting app.py


In [16]:
from pyngrok import ngrok
from google.colab import userdata
import time

# プロセスのリセット
ngrok.kill()

# --- Ngrokトークンの取得（シークレット推奨）---
try:
    NGROK_AUTH_TOKEN = userdata.get('NGROK_AUTH_TOKEN')
except:
    # シークレットがない場合はここに直接貼り付けてもOK
    NGROK_AUTH_TOKEN = "ここにあなたのNGROK_AUTH_TOKENを貼り付け"
# ----------------------------------------------

if not NGROK_AUTH_TOKEN or NGROK_AUTH_TOKEN == "ここにあなたのNGROK_AUTH_TOKENを貼り付け":
    print("⚠️ 警告: Ngrokトークンが正しく設定されていません。")
else:
    ngrok.set_auth_token(NGROK_AUTH_TOKEN)

    # Streamlitをバックグラウンド実行
    !streamlit run app.py &>/dev/null&

    time.sleep(3)

    try:
        public_url = ngrok.connect(8501)
        print(f"--------------------------------------------------")
        print(f"龍馬アプリが起動したぜよ！")
        print(f"URL: {public_url}")
        print(f"--------------------------------------------------")
    except Exception as e:
        print(f"エラー: {e}")


--------------------------------------------------
龍馬アプリが起動したぜよ！
URL: NgrokTunnel: "https://coccal-nonperceiving-anitra.ngrok-free.dev" -> "http://localhost:8501"
--------------------------------------------------
